In [1]:
import os
import numpy as np
import cv2
from collections import deque
import matplotlib.pyplot as plt
from tqdm import tqdm

def process_image(img_path, output_path, legend_width=150):

    img = cv2.imread(img_path)


    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


    height, width, _ = img.shape


    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    green_lower = np.array([35, 100, 100])
    green_upper = np.array([85, 255, 255])
    yellow_lower = np.array([25, 100, 100])
    yellow_upper = np.array([35, 255, 255])
 
    mask_green = cv2.inRange(hsv, green_lower, green_upper)
    mask_yellow = cv2.inRange(hsv, yellow_lower, yellow_upper)

    mask_combined = cv2.bitwise_or(mask_green, mask_yellow)

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

 
    def bfs(img, x, y, mask_combined):
        queue = deque([(x, y)])
        green_count = 0
        yellow_count = 0
        while queue:
            cx, cy = queue.popleft()
            if mask_combined[cy, cx] != 0:
                if np.array_equal(img[cy, cx], [0, 255, 0]):
                    green_count += 1
                elif np.array_equal(img[cy, cx], [255, 255, 0]):
                    yellow_count += 1
                mask_combined[cy, cx] = 0
                for dx, dy in directions:
                    nx, ny = cx + dx, cy + dy
                    if 0 <= nx < width and 0 <= ny < height and mask_combined[ny, nx] != 0:
                        queue.append((nx, ny))
        return green_count, yellow_count

  
    for y in range(height):
        for x in range(width - legend_width):
            if mask_combined[y, x] != 0:
                green_count, yellow_count = bfs(img, x, y, mask_combined.copy())
                if green_count >= yellow_count:
                    img[y, x] = [0, 255, 0]
                elif yellow_count > green_count:
                    img[y, x] = [255, 255, 0]


    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, img)


def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in tqdm(os.listdir(input_folder)):
        if filename.startswith('predicted_mask'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            process_image(input_path, output_path)
           # print(f"Processed {filename}")

for i in range(3):
    
    folder = f'./new_result/{13+i}'
    #output_folder = './processed_res'


    process_folder(folder,folder)


 89%|████████▉ | 481/540 [9:32:39<1:09:02, 70.20s/it]  